# TensorFlow dataset API and TensorFlow keras

In [1]:
import csv
import tensorflow as tf

## read the iris dataset

In [2]:
X = []
y = []
with open('/Users/zetong/IRIS.csv', newline='') as csvfile:
    iris = csv.reader(csvfile, delimiter=' ', quotechar='|')
    next(iris)
    for row in iris:
        row = row[0].split(",")
        X.append(row[0:-1])
        y.append(row[-1])

## label encoding

In [3]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)

LabelEncoder()

In [4]:
le.classes_

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype='<U15')

In [5]:
y = le.transform(y)

## divide train-test sets

In [6]:
import numpy as np
batch_size = 16
X = np.asarray(X)
X = X.astype(float)
y = np.asarray(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

## construct tf.keras model

In [30]:
i = tf.keras.layers.Input(shape=(X.shape[1]), name="input")
mlp = tf.keras.layers.Dense(16, name="mlp")(i)
out = tf.keras.layers.Dense(3, activation="softmax", name="output")(mlp)
model = tf.keras.Model(i, out)
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"])
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 4)]               0         
_________________________________________________________________
mlp (Dense)                  (None, 16)                80        
_________________________________________________________________
output (Dense)               (None, 3)                 51        
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


## creat tf.data.Dataset objects for training and testing

In [42]:
train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(16).repeat()
test = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(16).repeat()

## fitting the model

In [43]:
model.fit(train, steps_per_epoch=200, epochs=5)

W0908 00:33:07.320971 4591969728 training_utils.py:1300] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


Epoch 1/5
200/200 [==============================] - 0s 1ms/step - loss: 0.1286 - acc: 0.9634
Epoch 2/5
200/200 [==============================] - 0s 782us/step - loss: 0.1122 - acc: 0.9680
Epoch 3/5
200/200 [==============================] - 0s 730us/step - loss: 0.1011 - acc: 0.9744
Epoch 4/5
200/200 [==============================] - 0s 774us/step - loss: 0.0915 - acc: 0.9780
Epoch 5/5
200/200 [==============================] - 0s 733us/step - loss: 0.0846 - acc: 0.9779


## model evaluation

In [47]:
model.evaluate(test, steps=20)

20/20 [==============================] - 0s 4ms/step - loss: 0.0682 - acc: 1.0000


[0.06816048920154572, 1.0]